## Calculate ERPs

In [ ]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from os import path
%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data

## User variables

In [ ]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [717 , 768]
data_to_use = [768, 1536]
n_perm = 200

# Compute ERPs and Stats

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_learningOK/')
elec = 32

# file to compute
filename = 'CHAF_E1E2_concat_learning_0_bipo.npz'
data_all = np.load(path.join(path_data, filename))
data, channel, label = data_all['x'], data_all['channel'], data_all['label']

# Select data for one elec + name :
data_elec = data[elec,:,:]
ntrials = len(data_elec[2])
print ('\nOriginal data : ', data.shape, 'Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 'One elec shape : ', data_elec.shape)

#Filter data for one elec (all trials):
data = np.array(data_elec, dtype='float64')
data_to_filter = np.swapaxes(data, 0, 1)
filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
filtered_data = np.swapaxes(filtered_data, 0, 1)
print ('Size of filtered data:', filtered_data.shape,)

#Normalize the non-averaged data (all trials)
times = np.arange(filtered_data.shape[0])
print ('time points : ', times.shape)
filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
print ('Size norm & filtered data : ', norm_filtered_data.shape,)

#Generate the 3 learning files
learning_files = np.array_split(norm_filtered_data, 3, axis=1)
print ('learning file size 0', learning_files[0].shape, 'learning file size 1', learning_files[1].shape,'learning file size 2', learning_files[2].shape,)

# =======================================  STATISTICS  =====================================
# Range of the data to compute
data_range = range(data_to_use[0], data_to_use[1])

# Get all three learning files (Filtered data)
data_learn_0 = learning_files[0][data_range, :]
data_learn_1 = learning_files[1][data_range, :]
data_learn_2 = learning_files[2][data_range, :]
print ('-> Shape of the selected data for learn 0', data_learn_0.shape, 'learn 1', data_learn_1.shape, 'learn 2', data_learn_2.shape)

# Swap data across trials (dim 1) :
perm_learn_0_1 = perm_swap(data_learn_0, data_learn_1, axis=1, n_perm=n_perm)[0]
perm_learn_0_2 = perm_swap(data_learn_0, data_learn_2, axis=1, n_perm=n_perm)[0]
perm_learn_1_2 = perm_swap(data_learn_1, data_learn_2, axis=1, n_perm=n_perm)[0]
print('-> Shape of permuted learn 0/1 : ', perm_learn_0_1.shape, 'learn 0/2 : ', perm_learn_0_2.shape, 'learn 1/2 : ', perm_learn_1_2.shape)

# Take the mean across time :
perm_learn_0_1_mean = perm_learn_0_1.mean(2)
perm_learn_0_2_mean = perm_learn_0_2.mean(2)
perm_learn_1_2_mean = perm_learn_1_2.mean(2)
print('-> Shape of meaned permuted learn 0/1 : ', perm_learn_0_1_mean.shape, 'learn 0/2 : ', perm_learn_0_2_mean.shape, 'learn 1/2 : ', perm_learn_1_2_mean.shape)

# Get p-values from the permuted data :
p_vals_0_1 = perm_2pvalue(data_learn_0.mean(1), perm_learn_0_1_mean, n_perm=n_perm, threshold=None, tail=2)
p_vals_0_2 = perm_2pvalue(data_learn_2.mean(1), perm_learn_0_2_mean, n_perm=n_perm, threshold=None, tail=2)
p_vals_1_2 = perm_2pvalue(data_learn_1.mean(1), perm_learn_1_2_mean, n_perm=n_perm, threshold=None, tail=2)
print('-> Shape of p-values learn 0/1 : ', p_vals_0_1.shape, 'learn 0/2 : ', p_vals_0_2.shape, 'learn 1/2 : ', p_vals_1_2.shape)

## Test if there's significant p-values after multiplt comparison :
print('-> Significant p-values learn 0/1? ', p_vals_0_1.min() <= 0.05)
print('-> Significant p-values learn 0/2? ', p_vals_0_2.min() <= 0.05)
print('-> Significant p-values learn 1/2? ', p_vals_1_2.min() <= 0.05)

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
# Data to plot :
data_learn_0_to_plot = learning_files[0][range(baseline[0], data_to_use[1])]
data_learn_1_to_plot = learning_files[1][range(baseline[0], data_to_use[1])]
data_learn_2_to_plot = learning_files[2][range(baseline[0], data_to_use[1])]
data_learn_to_plot = np.concatenate([data_learn_0_to_plot, data_learn_1_to_plot, data_learn_2_to_plot], axis=1)
print('-> Shape of filtered data to plot : ', data_learn_to_plot.shape)

# Time vector & label vector:
times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
print('-> Shape of time vector : ', times_plot.shape)
label_learn_0 = np.zeros(data_learn_0_to_plot[1].shape, dtype='int64')
label_learn_1 = np.ones(data_learn_1_to_plot[1].shape, dtype='int64')
label_learn_2 = np.full(data_learn_2_to_plot[1].shape, 2, dtype='int64')
print('label size and values : 0 : ', label_learn_0.shape, '1 : ', label_learn_1.shape, '2 : ', label_learn_2.shape,)
y = np.concatenate([label_learn_0, label_learn_1, label_learn_2])
print('-> the y label', y.shape)

# P-values to plot :
p_vals_0_1_to_plot = np.insert(p_vals_0_1, 0, 10 * np.ones((baseline[1] - baseline[0],)))
p_vals_0_2_to_plot = np.insert(p_vals_0_2, 0, 10 * np.ones((baseline[1] - baseline[0],)))
p_vals_1_2_to_plot = np.insert(p_vals_1_2, 0, 10 * np.ones((baseline[1] - baseline[0],)))
print('-> Shape of p-values to plot :', p_vals_0_1_to_plot.shape, p_vals_0_2_to_plot.shape, p_vals_1_2_to_plot.shape)

#Prepare the plot
fig = plt.figure(0, figsize=(12, 7))
ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)
ax.set_xlabel('Times (ms)', fontsize=12)
ax.set_ylabel('Potential', fontsize=12)

#Plot the Data
BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.2, color=['m','c', 'b'], linewidth=2, ncol=1, legend= ['Early Learning', 'Medium Learning', 'Late Learning'],
          title='CHAF_ERP_Odor_bipo_'+norm_mode+'_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_ntrials:'+str(ntrials),)
plt.gca()
lines = [0] #time vector is in ms
addPval(plt.gca(), p_vals_0_1_to_plot, p=0.05, x=times_plot, y=0.5, color='m', lw=3)
addPval(plt.gca(), p_vals_0_2_to_plot, p=0.05, x=times_plot, y=1, color='b', lw=3)
addPval(plt.gca(), p_vals_1_2_to_plot, p=0.05, x=times_plot, y=1.5, color='c', lw=3)
addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
plt.legend(fontsize='small')
plt.grid()
plt.show()         


# Plot all ERPs with stats for learning cond

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250/')

subjects = ['CHAF','VACJ','SEMC','PIRJ','LEFC','MICP',]

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}

for su in subjects:
    for elec in range(0, n_elec[su],1):
        filename = su+'_E1E2_concat_allfilter1.npz'
        print (filename)
        data_all = np.load(path.join(path_data, filename))
        #data, channel, label = data_all['x'], data_all['channel'], data_all['label']
        data, channel = data_all['x'], [data_all['channel'][i][0] for i in range(len(data_all['channel']))]
        label = [data_all['label'][i][0] for i in range(len(data_all['label']))]

        # Select data for one elec + name :
        data_elec = data[elec,:,:]
        ntrials = len(data_elec[2])
        print ('\nOriginal data : ', data.shape, 'Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 'One elec shape : ', data_elec.shape)

        #Filter data for one elec (all trials):
        data = np.array(data_elec, dtype='float64')
        data_to_filter = np.swapaxes(data, 0, 1)
        filtered_data = filter_data(data_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        filtered_data = np.swapaxes(filtered_data, 0, 1)
        print ('Size of filtered data:', filtered_data.shape,)

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data.shape[0])
        print ('time points : ', times.shape)
        filtered_data_to_norm = np.swapaxes(filtered_data, 0, 1)
        norm_filtered_data = rescale(filtered_data_to_norm, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data = np.swapaxes(norm_filtered_data, 0, 1)
        print ('Size norm & filtered data : ', norm_filtered_data.shape,)

        #Generate the 3 learning files
        learning_files = np.array_split(norm_filtered_data, 3, axis=1)
        print ('learning file size 0', learning_files[0].shape, 'learning file size 1', learning_files[1].shape,'learning file size 2', learning_files[2].shape,)

# =======================================  STATISTICS  =====================================
        # Range of the data to compute
        data_range = range(data_to_use[0], data_to_use[1])

        # Get all three learning files (Filtered data)
        data_learn_0 = learning_files[0][data_range, :]
        data_learn_1 = learning_files[1][data_range, :]
        data_learn_2 = learning_files[2][data_range, :]
        print ('-> Shape of the selected data for learn 0', data_learn_0.shape, 'learn 1', data_learn_1.shape, 'learn 2', data_learn_2.shape)

        # Swap data across trials (dim 1) :
        perm_learn_0_1 = perm_swap(data_learn_0, data_learn_1, axis=1, n_perm=n_perm)[0]
        perm_learn_0_2 = perm_swap(data_learn_0, data_learn_2, axis=1, n_perm=n_perm)[0]
        perm_learn_1_2 = perm_swap(data_learn_1, data_learn_2, axis=1, n_perm=n_perm)[0]
        print('-> Shape of permuted learn 0/1 : ', perm_learn_0_1.shape, 'learn 0/2 : ', perm_learn_0_2.shape, 'learn 1/2 : ', perm_learn_1_2.shape)

        # Take the mean across time :
        perm_learn_0_1_mean = perm_learn_0_1.mean(2)
        perm_learn_0_2_mean = perm_learn_0_2.mean(2)
        perm_learn_1_2_mean = perm_learn_1_2.mean(2)
        print('-> Shape of meaned permuted learn 0/1 : ', perm_learn_0_1_mean.shape, 'learn 0/2 : ', perm_learn_0_2_mean.shape, 'learn 1/2 : ', perm_learn_1_2_mean.shape)

        # Get p-values from the permuted data :
        p_vals_0_1 = perm_2pvalue(data_learn_0.mean(1), perm_learn_0_1_mean, n_perm=n_perm, threshold=None, tail=2)
        p_vals_0_2 = perm_2pvalue(data_learn_2.mean(1), perm_learn_0_2_mean, n_perm=n_perm, threshold=None, tail=2)
        p_vals_1_2 = perm_2pvalue(data_learn_1.mean(1), perm_learn_1_2_mean, n_perm=n_perm, threshold=None, tail=2)
        print('-> Shape of p-values learn 0/1 : ', p_vals_0_1.shape, 'learn 0/2 : ', p_vals_0_2.shape, 'learn 1/2 : ', p_vals_1_2.shape)

        ## Test if there's significant p-values after multiplt comparison :
        print('-> Significant p-values learn 0/1? ', p_vals_0_1.min() <= 0.05)
        print('-> Significant p-values learn 0/2? ', p_vals_0_2.min() <= 0.05)
        print('-> Significant p-values learn 1/2? ', p_vals_1_2.min() <= 0.05)

        # =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
        # Data to plot :
        data_learn_0_to_plot = learning_files[0][range(baseline[0], data_to_use[1])]
        data_learn_1_to_plot = learning_files[1][range(baseline[0], data_to_use[1])]
        data_learn_2_to_plot = learning_files[2][range(baseline[0], data_to_use[1])]
        data_learn_to_plot = np.concatenate([data_learn_0_to_plot, data_learn_1_to_plot, data_learn_2_to_plot], axis=1)
        print('-> Shape of filtered data to plot : ', data_learn_to_plot.shape)

        # Time vector & label vector:
        times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
        print('-> Shape of time vector : ', times_plot.shape)
        label_learn_0 = np.zeros(data_learn_0_to_plot[1].shape, dtype='int64')
        label_learn_1 = np.ones(data_learn_1_to_plot[1].shape, dtype='int64')
        label_learn_2 = np.full(data_learn_2_to_plot[1].shape, 2, dtype='int64')
        print('label size and values : 0 : ', label_learn_0.shape, '1 : ', label_learn_1.shape, '2 : ', label_learn_2.shape,)
        y = np.concatenate([label_learn_0, label_learn_1, label_learn_2])
        print('-> the y label', y.shape)

        # P-values to plot :
        p_vals_0_1_to_plot = np.insert(p_vals_0_1, 0, 10 * np.ones((baseline[1] - baseline[0],)))
        p_vals_0_2_to_plot = np.insert(p_vals_0_2, 0, 10 * np.ones((baseline[1] - baseline[0],)))
        p_vals_1_2_to_plot = np.insert(p_vals_1_2, 0, 10 * np.ones((baseline[1] - baseline[0],)))
        print('-> Shape of p-values to plot :', p_vals_0_1_to_plot.shape, p_vals_0_2_to_plot.shape, p_vals_1_2_to_plot.shape)

        #Prepare the plot
        fig = plt.figure(0, figsize=(12, 7))
        ax = fig.add_subplot(111)
        fig.subplots_adjust(top=0.85)
        ax.set_xlabel('Times (ms)', fontsize=12)
        ax.set_ylabel('Potential', fontsize=12)

        #Plot the Data
        BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.2, color=['m','c', 'b'], linewidth=2, ncol=1, legend= ['Early Learning', 'Medium Learning', 'Late Learning'],
                  title=su+'_ERP_Odor_mono_'+norm_mode+'_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_ntrials:'+str(ntrials),)
        plt.gca()
        lines = [0] #time vector is in ms
        addPval(plt.gca(), p_vals_0_1_to_plot, p=0.05, x=times_plot, y=0.5, color='m', lw=3)
        addPval(plt.gca(), p_vals_0_2_to_plot, p=0.05, x=times_plot, y=1, color='b', lw=3)
        addPval(plt.gca(), p_vals_1_2_to_plot, p=0.05, x=times_plot, y=1.5, color='c', lw=3)
        addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
        plt.legend(fontsize='small')
        plt.grid()
        #plt.show()     

# =========================SAVE PLOTS of ERPs=================================================
        rep = path.join(st.path, 'feature/ERP_Encoding_all_mono_100ms_mean_thr40_art400_30_250_learning/',su)
        fname = (rep + '_E1E2_ERP_concat_all_mono_' + channel [elec] +'_'+str(elec)+'_'+label[elec]+'.png')
        print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.close()
del x, channel, n_elec, n_trials, label